In [ ]:
from google.colab import files
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import textwrap
import docx
import PyPDF2

      # Function to read text from different file types
def read_file(file_path):
    if file_path.lower().endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()

    elif file_path.lower().endswith(".docx"):
        doc = docx.Document(file_path)
        return "\n".join([p.text for p in doc.paragraphs])

    elif file_path.lower().endswith(".pdf"):
        text = ""
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() + "\n"
        return text

    else:
        raise ValueError("Unsupported file type. Please use TXT, DOCX, or PDF.")




              # Choose input method
choice = input("📌 Enter '1' to upload a file or '2' to input text manually: ")

if choice == "1":
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    text = read_file(file_path)
elif choice == "2":
    print("✏️ Enter the text, then press Enter twice:")
    lines = []
    while True:
        line = input()
        if line == "":
            break
        lines.append(line)
    text = "\n".join(lines)
else:
    raise ValueError("Invalid choice")


      # 🤖 English summarization using BART
bart_name = "facebook/bart-large-cnn"
bart_tokenizer = AutoTokenizer.from_pretrained(bart_name)
bart_model = AutoModelForSeq2SeqLM.from_pretrained(bart_name)

bart_inputs = bart_tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
bart_summary_ids = bart_model.generate(
    bart_inputs["input_ids"],
    max_length=150,
    min_length=40,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)
bart_summary = bart_tokenizer.decode(bart_summary_ids[0], skip_special_tokens=True)

print("\n🔹 English Summary:\n", bart_summary)

📌 Enter '1' to upload a file or '2' to input text manually: 2
✏️ Enter the text, then press Enter twice:
AAAaaaaaaaaaaaaaasdfghjklm,lkmknjnjnjnjnjnuhi


🔹 English Summary:
 AAAaaaaaaaaaaaaaasdfghjklm,lkmknjnjn Jnjnuhi, lkmkn JnJnuhi. LkmknJnJn JNnuhi,.lkmKnJnjNjNuhi, LkmKnNjnJNnuHI. LkknJNnJnihi,lkknNjnihi.


In [ ]:

# --- ❓ Question Answering ---

qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

def answer_question_over_chunks(question, context, chunk_size=500):
    chunks = textwrap.wrap(context, chunk_size)
    best_answer = {"score": 0, "answer": ""}
    for chunk in chunks:
        try:
            ans = qa_pipeline(question=question, context=chunk)
            if ans["score"] > best_answer["score"]:
                best_answer = ans
        except:
            continue
    return best_answer

while True:
    question = input("\n💬 اكتب سؤالك (أو اكتب exit للخروج): ")
    if question.lower() == "exit":
        break

    # Search in the summary
    ans_summary = qa_pipeline(question=question, context=bart_summary)

    # Search in the full text (chunked)
    ans_full = answer_question_over_chunks(question, text)

    # Select the best answer
    if ans_summary["score"] > ans_full["score"]:
        best_ans = ans_summary
        source = "📄 from  summary"
    else:
        best_ans = ans_full
        source = "📚 from full text "

    print(f"📝 الإجابة: {best_ans['answer']}  (score: {best_ans['score']:.2f}) {source}")

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0



💬 اكتب سؤالك (أو اكتب exit للخروج): What does Artificial Intelligence (AI) refer to?
📝 الإجابة: simulation of human intelligence in machines  (score: 0.44) 📄 from  summary

💬 اكتب سؤالك (أو اكتب exit للخروج): Name three subfields of AI mentioned in the article.
📝 الإجابة: machine learning, natural language processing, computer vision, and robotics  (score: 0.81) 📚 from full text 

💬 اكتب سؤالك (أو اكتب exit للخروج): How does AI benefit the healthcare industry?


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


📝 الإجابة: raises ethical concerns  (score: 0.11) 📄 from  summary

💬 اكتب سؤالك (أو اكتب exit للخروج): exit


In [ ]:
%pip install python-docx PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.9 MB/s eta 0:00:00


After installing the necessary libraries, you can now run the previous cell to read the text from your file and summarize it.